# Step I. ETL Pipeline for Pre-Processing the Files

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import re
import os
import glob
import numpy as np
import json
import csv
import cassandra
import re

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking current working directory
# print(os.getcwd())

In [4]:
# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    if (re.search(".*event_data$", root)):
    # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

In [6]:
# get total number of rows 
print(len(full_data_rows_list))
# check to see what the list of event data rows will look like
# print(full_data_rows_list)

8056


In [7]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [8]:
# check the number of rows in csv file event_datafile_new
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Step II. Modeling the Apache Cassandra database

## Work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the file event_datafile_new.csv after the code above is run:<br>

<img src="images/event_datafile_new.png" style="width:2000px;">

#### Creating a Cluster

In [9]:
# This makes a connection to a Cassandra instance in local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
try:
    session.execute("create Keyspace if not exists sparkify with replication = {'class': 'SimpleStrategy', 'replication_factor': 1}")
except Exception as e:
    print(e)

#### Set Keyspace

In [11]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

#### Helper Functions

In [12]:
def insert_data(query, value_index_list, value_type_list, session):
    file = 'event_datafile_new.csv'

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple(value_type(line[index]) for index, value_type in zip(value_index_list, value_type_list)))

In [13]:
def generate_result(query, session):
    result = []
    
    try:
        rows = session.execute(query)
        for row in rows:
            result.append(row)
    except Exception as e:
        print(e)

    return result

### Query: show the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [14]:
# create table
query = "create table if not exists songs_by_sessionId_itemInSession"
query = query + " (sessionId int, itemInSession int, artist_name varchar, song_title varchar, song_length float, primary key (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)              


In [15]:
# insert data
query = "insert into songs_by_sessionId_itemInSession (sessionId, itemInSession, artist_name, song_title, song_length) values (%s, %s, %s, %s, %s)"
value_index_list = [8, 3, 0, 9, 5]
value_type_list = [int, int, str, str, float]

insert_data(query, value_index_list, value_type_list, session)


In [16]:
# generate result
query = "select artist_name, song_title, song_length from songs_by_sessionId_itemInSession where sessionId = 338 and itemInSession = 4"
result = generate_result(query, session)


In [17]:
# present result
df = pd.DataFrame(result)
df.columns = ["artist_name", "song_title", "song_length"]
df

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query: show the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [18]:
# create table
query = "create table if not exists history_by_userid_sessionid"
query = query + " (sessionId int, userId int, itemInSession int, song_title varchar, artist_name varchar, user_first_name varchar, user_last_name varchar, primary key ((sessionId, userId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)              


In [19]:
# insert data
query = "insert into history_by_userid_sessionid (sessionId, userId, itemInSession, song_title, artist_name, user_first_name, user_last_name) values (%s, %s, %s, %s, %s, %s, %s)"
value_index_list = [8, 10, 3, 9, 0, 1, 4]
value_type_list = [int, int, int, str, str, str, str]

insert_data(query, value_index_list, value_type_list, session)

In [20]:
# generate result
query = "select song_title, artist_name, user_first_name, user_last_name from history_by_userid_sessionid where sessionId = 182 and userId = 10"
result = generate_result(query, session)


In [21]:
# present result
df = pd.DataFrame(result)
df.columns = ["song_title", "artist_name", "user_first_name", "user_last_name"]
df

,song_title,artist_name,user_first_name,user_last_name
0,Keep On Keepin' On,Down To The Bone,Sylvie,Cruz
1,Greece 2000,Three Drives,Sylvie,Cruz
2,Kilometer,Sebastien Tellier,Sylvie,Cruz
3,Catch You Baby (Steve Pitron & Max Sanna Radio...,Lonnie Gordon,Sylvie,Cruz


### Query: show user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

In [22]:
# create table
query = "create table if not exists users_by_songs"
query = query + " (song_title varchar, user_first_name varchar, user_last_name varchar, userId int, primary key (song_title, user_first_name, user_last_name, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)   
                    

In [23]:
# insert data
query = "insert into users_by_songs (song_title, user_first_name, user_last_name, userId) values (%s, %s, %s, %s)"
value_index_list = [9, 1, 4, 10]
value_type_list = [str, str, str, int]

insert_data(query, value_index_list, value_type_list, session)

In [24]:
# generate result
query = "select user_first_name, user_last_name from users_by_songs where song_title = 'All Hands Against His Own'"
result = generate_result(query, session)

In [25]:
# present result
df = pd.DataFrame(result)
df.columns = ["user_first_name", "user_last_name"]
df

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [26]:
try:
    session.execute("drop table if exists users_by_songs")
except Exception as e:
    print(e)

In [27]:
try:
    session.execute("drop table if exists history_by_userid_sessionid")
except Exception as e:
    print(e)

In [28]:
try:
    session.execute("drop table if exists songs_by_sessionId_itemInSession")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [29]:
session.shutdown()
cluster.shutdown()